In [1]:
%pylab
import os, re, time, sys
from PIL import Image

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# This part deals with setting the paths for each computer / os

# Linux and Windows have different syntax for file paths and we want compatibility with both
# So, file paths are stored in a file called:
# declic_settings.txt
# located in the SAME directory/folder as this .ipynb file
# The local file should contain 2 paths set up in the following way:
# save_path = 'insert directory you want to save to here'
# im_path = 'insert directory your images are coming from here'
# On Windows, you need to represent \ in a file path as \\
# Saves correct path strings in variables called:
# save_path, im_path

# Copy experimental parameters corresponding to experiment of images used from separate file called:
# exp_params.txt
# located in the SAME directory/folder as this .ipynb file
# updated to reflect accurate values from the DECLIC experiment file
# The local file should contain 3 lines set up in the following way:
# moteur = 9-10-digit positive integer goes here (column L in manips file for exp line)
# start_tick = 9-10-digit positive integer > moteur goes here (column R in manips file for exp line)
# end_tick = 9-10 digit positive integer > start_tick goes here (column O in manips file for exp line)
# Do not use any quotation marks or apostrophes around the integer values
# Saves values to variables called:
# moteur, start_tick, end_tick

from sys import platform
if platform == "linux" or platform == "linux2":
    path = os.path.abspath('.')
    file_settings_path = "./declic_settings_linux.cfg"
    save_path, im_path = False, False
    f = open(file_settings_path,'r')
    f = list(f)
    for line in f:
        try:
            line = line.replace(' ','').replace('\n','').replace("'","").replace('"','')
            line = line.split('=')
            varname, varval = line
            varname = varname.replace('\n','')
            varval = varval.replace('\n','').replace("'","")
            if varname == "save_path": save_path = varval
            elif varname == "im_path": im_path = varval
        except:
            print sys.exc_info()
            print "bad path definition file or not found!"
    if not (save_path and im_path): print "error reading path variables"
    
    exp_param_path = "./exp_params.txt"
    moteur, start_tick, end_tick = 0, 0, 0
    p = open(exp_param_path, 'r')
    p = list(p)
    for line in p:
        try:
            line = line.replace(' ','').replace('\n','')
            line = line.split('=')
            varname, varval = line
            varname = varname.replace('\n','')
            varval = varval.replace('\n','')
            if varname == "moteur": moteur = int(varval)
            elif varname == "start_tick": start_tick = int(varval)
            elif varname == "end_tick": end_tick = int(varval)
        except:
            print sys.exc_info()
            print "bad path definition file or not found!"
    if not (moteur > 0 and start_tick > 0 and end_tick > 0): print "error reading path variables"
        
elif platform == "darwin":
    settings_path = "~/declic_settings_mac.cfg"
    print "Mac implementation not handled in current code"
    
elif platform == "win32":
    path = os.path.abspath('.')
    file_settings_path = os.path.expanduser(path)+u'\\declic_settings_windows.txt'
    save_path, im_path = False, False
    f = open(file_settings_path,'r')
    f = list(f)
    for line in f:
        try:
            line = line.replace(' ','').replace('\n','').replace("'","").replace('"','')
            line = line.replace('\\\\','\\').split('=') #extra \\ requirement for windows path
            varname, varval = line
            varname = varname.replace('\n','')
            varval = varval.replace('\n','').replace("'","")
            if varname == "save_path": save_path = varval
            elif varname == "im_path": im_path = varval
        except:
            print sys.exc_info()
            print "bad path definition file or not found!"
    if not (save_path and im_path): print "error reading path variables"
    
    exp_param_path = "./exp_params.txt"
    moteur, start_tick, end_tick = 0, 0, 0
    p = open(exp_param_path, 'r')
    p = list(p)
    for line in p:
        try:
            line = line.replace(' ','').replace('\n','')
            line = line.split('=')
            varname, varval = line
            varname = varname.replace('\n','')
            varval = varval.replace('\n','')
            if varname == "moteur": moteur = int(varval)
            elif varname == "start_tick": start_tick = int(varval)
            elif varname == "end_tick": end_tick = int(varval)
        except:
            print sys.exc_info()
            print "bad path definition file or not found!"
    if not (moteur > 0 and start_tick > 0 and end_tick > 0): print "error reading path variables"
        

In [5]:
def imagesavewithtimer(all_names):
    # sequentially open files in a local directory and save to another local directory 
    for n in range(len(all_names)):
        thisname = all_names[n]
        chrono = time.time()
        I = np.array(Image.open(thisname))/255.0
        plt.figure()
        plt.imshow(I)
        plt.savefig('{:s}image{:03d}_.png'.format(save_path,n), dpi=300)
        plt.close()
        chrono = time.time()-chrono
        if n%10==0: #for every 10th image, print the estimated processing time per image and until completion
            print 'treating {:d} of {:d} images, at {:0.1f} seconds per image. Approx {:0.1f} minutes remaining'.format(n, len(all_names), chrono, (len(all_names)-n)*chrono/60)
 
names = os.listdir(im_path)#make a list of files
names = [i for i in names if '1.bmp' in i]
names.sort() #need to put in order
temp_names = []
times = []
    
for n in names: #step through list and save tick 
    try:
        tick = int(re.findall('[0-9]{9,10}',n)[0]) #9-10 numeric characters
        if tick >= start_tick:
            temp_names.append(n)
            times.append((tick - moteur)/23.0) #second diff, incl prime number in system
        if tick > end_tick: break
    except: continue
names = ['{}{}'.format(im_path,n) for n in temp_names] #assigns number related to order in names

times = np.array(times)

In [6]:
imagesavewithtimer(names) #run the image opening and saving function

treating 0 of 258 images, at 1.7 seconds per image. Approx 7.2 minutes remaining
treating 10 of 258 images, at 1.8 seconds per image. Approx 7.3 minutes remaining
treating 20 of 258 images, at 1.7 seconds per image. Approx 6.8 minutes remaining


KeyboardInterrupt: 